## Configurando o ambiente

In [1]:
!pip install google-adk -q
!pip install litellm -q
!pip install dpip install ddgs -q

print("Installation complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 21.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement dpip (from versions: none)
ERROR: No matching distribution found for dpip
Installation complete.


In [2]:
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
import math
from google.adk.tools import FunctionTool


import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [3]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

API Keys Set:
Google API Key set: Yes
OpenAI API Key set: Yes
Anthropic API Key set: Yes


In [4]:
MODEL_GEMINI_2_5_FLASH = "gemini-2.5-flash"
MODEL_GPT_4O = "openai/gpt-4.1-nano"
MODEL_CLAUDE_SONNET = "anthropic/claude-sonnet-4-5-20250929"

print("\nEnvironment configured.")


Environment configured.


## Definindo Sub Agentes e Ferramentas

### Ferramentas

In [5]:
# @title Ferramenta de cálculo
def physics_calculator(expression: str) -> str:
    """
    Avalia expressões matemáticas de forma segura para cálculos de física.

    O agente deve usar esta ferramenta para qualquer operação matemática.
    Aceita tanto chamadas diretas (ex: 'sqrt(25)') quanto com prefixo (ex: 'math.sqrt(25)').

    Args:
        expression: A expressão matemática em string.
    """
    try:
        # 1. Prepara o dicionário de nomes permitidos (funções do math como 'sin', 'cos', 'sqrt')
        allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}

        # Isso permite que o eval entenda 'math.pi' ou 'math.sqrt'
        allowed_names['math'] = math

        result = eval(expression, {"__builtins__": {}}, allowed_names)

        return str(result)

    except Exception as e:
        return f"Erro ao calcular '{expression}': {str(e)}"

calculator_tool = FunctionTool(physics_calculator)

print("✅ Ferramenta 'physics_calculator' criada e configurada com sucesso.")

✅ Ferramenta 'physics_calculator' criada e configurada com sucesso.


In [6]:
# @title Testando a calculadora
print("=== INICIANDO TESTES DA CALCULADORA ===")

# Teste 1: Aritmética Básica
res_basico = physics_calculator("10 * 9.8")
print(f"1. Aritmética (10 * 9.8): \t{res_basico} \t(Esperado: 98.0)")

# Teste 2: Módulo Math com Prefixo (O que estava falhando antes)
res_prefixo = physics_calculator("math.sqrt(49)")
print(f"2. Prefixo 'math.sqrt(49)': \t{res_prefixo} \t(Esperado: 7.0)")

# Teste 3: Módulo Math sem Prefixo (Robustez)
res_sem_prefixo = physics_calculator("cos(0)")
print(f"3. Sem prefixo 'cos(0)': \t{res_sem_prefixo} \t(Esperado: 1.0)")

# Teste 4: Uso de Constantes e Funções Compostas
# Calcula circunferência: 2 * pi * r (r=5)
res_composto = physics_calculator("2 * math.pi * 5")
print(f"4. Composto (2 * pi * 5): \t{res_composto} \t(Esperado: ~31.41)")

# Teste 5: Segurança (Deve falhar)
input_malicioso = "__import__('os').listdir('.')"
res_seguranca = physics_calculator(input_malicioso)
print(f"5. Teste de Segurança: \t\t{res_seguranca}")

# Verificação final do Teste 5
if "name '__import__' is not defined" in res_seguranca or "Erro" in res_seguranca:
    print("\n✅ SUCESSO: O sistema bloqueou a injeção de código.")
else:
    print("\n❌ PERIGO: O sistema permitiu a injeção de código.")

=== INICIANDO TESTES DA CALCULADORA ===
1. Aritmética (10 * 9.8): 	98.0 	(Esperado: 98.0)
2. Prefixo 'math.sqrt(49)': 	7.0 	(Esperado: 7.0)
3. Sem prefixo 'cos(0)': 	1.0 	(Esperado: 1.0)
4. Composto (2 * pi * 5): 	31.41592653589793 	(Esperado: ~31.41)
5. Teste de Segurança: 		Erro ao calcular '__import__('os').listdir('.')': name '__import__' is not defined

✅ SUCESSO: O sistema bloqueou a injeção de código.


### Sub Agentes

In [7]:
# @title Agente Didático (Claude)
pedagogo_agent = Agent(
    name="agente_didatico",
    model=LiteLlm(model=MODEL_CLAUDE_SONNET),
    tools=[],
    instruction="""
        Você é um Professor de Física extremamente didático e empático (Persona: Carl Sagan).
        ENTRADA: Você receberá um JSON contendo dados técnicos (cálculos ou fatos).

        SUA MISSÃO:
        1. Ler o JSON.
        2. Explique a "formula_descritiva" em palavras simples (ex: "A velocidade depende da altura...").
        3. Use analogias para os "valores_input" (ex: "10 metros é como um prédio de 3 andares").
        4. Apresente o "resultado" final com clareza.

        TOM DE VOZ:
        - Use "Nós vamos descobrir...", "Imagine que...".
        - Valide o esforço do aluno.
        - NÃO mencione "JSON" ou "API". Aja naturalmente.
        """,
    description="Recebe informações técnicas (conceitos e cálculos) e transcreve-os de forma didática para crianças e adolencentes poderem entender mais facilmente.",
)

print(f"✅ Agente '{pedagogo_agent.name}' criado (Foco: Didática e Analogias).")

✅ Agente 'agente_didatico' criado (Foco: Didática e Analogias).


In [8]:
# @title Agente Técnico (GPT-4.1)
tecnico_agent = Agent(
    name="agente_tecnico",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="""
        Você é uma API de Cálculo Científico.
        Sua ÚNICA função é resolver problemas matemáticos usando a ferramenta 'physics_calculator', não faça buscas, use apenas suas ferramentas e encaminhe a reposta para o seu sub_agent.

        REGRA DE SAÍDA (IMPORTANTE):
        Você NÃO deve responder diretamente o usuário e não deve chamar ferramentas de busca.
        Você deve sempre calcular a resposta e enviar APENAS o objeto JSON válido para o pedagogo_agent, ele é o responsável pela explicação.
        Não coloque crases (```json), apenas o objeto bruto.

        Formato obrigatório do JSON:
        {
        "tipo": "calculo",
        "contexto": "Descreva brevemente o que foi calculado (ex: Velocidade de queda)",
        "formula_descritiva": "A fórmula usada (ex: v = sqrt(2*g*h))",
        "valores_input": "Quais valores foram usados (ex: h=10, g=9.8)",
        "resultado": "O número exato retornado pela ferramenta",
        "unidade": "A unidade física (ex: m/s, Joules, Newtons)"
        }
        """,
    description="Faz uso da ferramenta de cálculo para fazer operação matemáticas e encaminha as repostas em formato JSON para o pedagogo_agent.",
    tools=[calculator_tool],
    sub_agents=[pedagogo_agent]
)

print(f"✅ Agente '{tecnico_agent.name}' configurado com ferramentas: {[t.name for t in tecnico_agent.tools]}")

✅ Agente 'agente_tecnico' configurado com ferramentas: ['physics_calculator']


In [9]:
# @title Agente de Busca (Gemini)

from google.adk.tools import google_search

search_agent = Agent(
    name="agente_de_buscas",
    model=MODEL_GEMINI_2_5_FLASH,
    tools=[google_search],
    instruction="""
        Você é uma API de Dados Históricos e Físicos.
        Sua função é usar o 'google_search' para encontrar fatos precisos.

        REGRA DE SAÍDA:
        Explique de forma simples e coesa o resultado da pesquisa.
        """,
    description="Faz buscas na internet e apresenta o resultado de forma simples",
)

print(f"✅ Agente '{search_agent.name}' .")

✅ Agente 'agente_de_buscas' .


## Testando Agentes

In [ ]:
# @title Testando o Agente Didático
# SIMULAÇÃO

print(f"--- ENTRADA (Do Técnico) ---\n")
print("--- PROCESSANDO (Pelo Pedagogo) ---\n")

await run_conversation()

## Definindo o Agente Root

In [10]:
# @title Agente Root
root_agent = Agent(
    name="Agente_Coordenador_Root",
    model=LiteLlm(model=MODEL_CLAUDE_SONNET),
    instruction="""
        Você é o Orquestrador do Sistema.

        FLUXO DE EXECUÇÃO RIGOROSO (PIPELINE):

        PASSO 1 (Triagem):
        - Analise a pergunta. Se não for de física/ciências, rejeite.

        PASSO 2 (Obtenção de Dados):
        - Se for CÁLCULO MATEMÁTICO: Chame 'tecnico_agent'.
        - Se for uma BUSCA POR FATOS OU PERSONALIDADES: Chame 'search_agent'.

        """,
    description="Coordenador do sistema. Filtra assuntos irrelevantes e gerencia o time.",
    sub_agents=[tecnico_agent, search_agent],
)

print(f"✅ Agente Root criado. Time completo: {[t.name for t in root_agent.tools]}")

✅ Agente Root criado. Time completo: []


In [11]:
# @title Runner e Session Service

session_service = InMemorySessionService()
APP_NAME = "equipe_agentes_monitores_fisica"
USER_ID = "user_1_agent_team"
SESSION_ID = "session_001_agent_team"
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)

runner_root = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print(f"Runner created for agent")

Runner created for agent


In [12]:
# @title Função de Chamada do Agente
from google.genai import types

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> Questão do usuário: {query}")

  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # logica do agente executada de forma assíncrona e produz eventos.
  # a interação é feita através dos eventos até a resposta final.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # motra todos os eventos durante a execução
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate:
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          break

  print(f"<<< Resposta do Agente: {final_response_text}")

async def run_conversation(input_question: str):
    await call_agent_async(input_question,
                                       runner=runner_root,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

In [15]:
# @title Testando o Agente Root

print("... 🤖 O Time de Agentes está trabalhando (Isso pode levar alguns segundos) ...\n")

input_question = input("Digite sua pergunta: ")

await run_conversation(input_question)

... 🤖 O Time de Agentes está trabalhando (Isso pode levar alguns segundos) ...

Digite sua pergunta: qual a velocidade de um movel em repouso que acelera por 5s a 2m/s² ?

>>> Questão do usuário: qual a velocidade de um movel em repouso que acelera por 5s a 2m/s² ?
  [Event] Author: agente_tecnico, Type: Event, Final: True, Content: parts=[Part(
  text="""{
  "tipo": "calculo",
  "contexto": "Velocidade final de um móvel que inicia em repouso e acelera",
  "formula_descritiva": "v = a * t",
  "valores_input": "a=2 m/s², t=5 s, velocidade inicial=0",
  "resultado": 10,
  "unidade": "m/s"
}"""
)] role='model'
<<< Resposta do Agente: {
  "tipo": "calculo",
  "contexto": "Velocidade final de um móvel que inicia em repouso e acelera",
  "formula_descritiva": "v = a * t",
  "valores_input": "a=2 m/s², t=5 s, velocidade inicial=0",
  "resultado": 10,
  "unidade": "m/s"
}


## Adicionar Proteções

In [35]:
# @title 1. Guardrail de Lista de Palavras

from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # For creating response content
from typing import Optional


def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Inspeciona a última mensagem do usuário buscando por QUALQUER palavra
    de uma lista proibida. Se encontrar, bloqueia a chamada ao LLM.
    """
    agent_name = callback_context.agent_name
    print(f"--- Callback: block_keyword_guardrail running for agent: {agent_name} ---")

    last_user_message_text = ""
    if llm_request.contents:
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break

    print(f"--- Callback: Inspecting: '{last_user_message_text[:50]}...' ---")


    blocked_words = [
        "BLOCK",
        "IGNORE",
        "HACK",
        "POLÍTICA",
        "VIOLÊNCIA",
        "PROMPT INJECTION"
    ]

    user_text_upper = last_user_message_text.upper()
    print(f"--- Callback: user_text_upper: '{user_text_upper}' ---") # Debug print

    for word in blocked_words:
        if word in user_text_upper:
            print(f"--- Callback: Found blocked word '{word}'. Blocking LLM call! ---")

            # Registra no estado qual palavra foi detectada
            callback_context.state["guardrail_triggered"] = True
            callback_context.state["blocked_word"] = word
            print(f"--- Callback: State updated. Blocked by: {word} ---")

            # Retorna a resposta de bloqueio imediato
            return LlmResponse(
                content=types.Content(
                    role="model",
                    parts=[types.Part(text=f"Não foi possível processar a requisição devido a palavra chave bloqueda: '{word}'.")],
                )
            )

    # Se o loop terminar sem retornar, nenhuma palavra foi encontrada
    print(f"--- Callback: Sem palavras bloqueadas. Uso da LLM permitido para {agent_name}. ---")
    return None

print("✅ Função de proteção do prompt atualiada com sucesso.")

✅ Função de proteção do prompt atualiada com sucesso.


In [36]:
# @title 2. Agente Orquestrador

root_agent = None
runner_root_model_guardrail = None

pedagogo_agent = Agent(
    name="agente_didatico",
    model=LiteLlm(model=MODEL_CLAUDE_SONNET),
    tools=[],
    instruction="""
        Você é um Professor de Física extremamente didático e empático (Persona: Carl Sagan).
        ENTRADA: Você receberá um JSON contendo dados técnicos (cálculos ou fatos).

        SUA MISSÃO:
        1. Ler o JSON.
        2. Explique a "formula_descritiva" em palavras simples (ex: "A velocidade depende da altura...").
        3. Use analogias para os "valores_input" (ex: "10 metros é como um prédio de 3 andares").
        4. Apresente o "resultado" final com clareza.

        TOM DE VOZ:
        - Use "Nós vamos descobrir...", "Imagine que...".
        - Valide o esforço do aluno.
        - NÃO mencione "JSON" ou "API". Aja naturalmente.
        """,
    description="Recebe informações técnicas (conceitos e cálculos) e transcreve-os de forma didática para crianças e adolencentes poderem entender mais facilmente.",
)

tecnico_agent = Agent(
    name="agente_tecnico",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="""
        Você é uma API de Cálculo Científico.
        Sua ÚNICA função é resolver problemas matemáticos usando a ferramenta 'physics_calculator', não faça buscas, use apenas suas ferramentas e encaminhe a reposta para o seu sub_agent.

        REGRA DE SAÍDA (IMPORTANTE):
        Você NÃO deve responder diretamente o usuário e não deve chamar ferramentas de busca.
        Você deve sempre calcular a resposta e enviar APENAS o objeto JSON válido para o pedagogo_agent, ele é o responsável pela explicação.
        Não coloque crases (```json), apenas o objeto bruto.

        Formato obrigatório do JSON:
        {
        "tipo": "calculo",
        "contexto": "Descreva brevemente o que foi calculado (ex: Velocidade de queda)",
        "formula_descritiva": "A fórmula usada (ex: v = sqrt(2*g*h))",
        "valores_input": "Quais valores foram usados (ex: h=10, g=9.8)",
        "resultado": "O número exato retornado pela ferramenta",
        "unidade": "A unidade física (ex: m/s, Joules, Newtons)"
        }
        """,
    description="Faz uso da ferramenta de cálculo para fazer operação matemáticas e encaminha as repostas em formato JSON para o pedagogo_agent.",
    tools=[calculator_tool],
    sub_agents=[pedagogo_agent]
)

from google.adk.tools import google_search

search_agent = Agent(
    name="agente_de_buscas",
    model=MODEL_GEMINI_2_5_FLASH,
    tools=[google_search],
    instruction="""
        Você é uma API de Dados Históricos e Físicos.
        Sua função é usar o 'google_search' para encontrar fatos precisos.

        REGRA DE SAÍDA:
        Explique de forma simples e coesa o resultado da pesquisa.
        """,
    description="Faz buscas na internet e apresenta o resultado de forma simples",
)

root_agent = Agent(
    name="Agente_Coordenador_Root",
    model=LiteLlm(model=MODEL_CLAUDE_SONNET),
    instruction="""
        Você é o Orquestrador do Sistema.

        FLUXO DE EXECUÇÃO RIGOROSO (PIPELINE):

        PASSO 1 (Triagem):
        - Analise a pergunta. Se não for de física/ciências, rejeite.

        PASSO 2 (Obtenção de Dados):
        - Se for CÁLCULO MATEMÁTICO: Chame 'tecnico_agent'.
        - Se for uma BUSCA POR FATOS OU PERSONALIDADES: Chame 'search_agent'.

        """,
    description="Coordenador do sistema. Filtra assuntos irrelevantes e gerencia o time.",
    sub_agents=[tecnico_agent, search_agent],
    before_model_callback=block_keyword_guardrail,
)



In [37]:
# @title Função de Chamada do Agente
from google.genai import types

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> Questão do usuário: {query}")

  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # logica do agente executada de forma assíncrona e produz eventos.
  # a interação é feita através dos eventos até a resposta final.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # motra todos os eventos durante a execução
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate:
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          break

  print(f"<<< Resposta do Agente: {final_response_text}")

async def run_conversation(input_question: str):
    await call_agent_async(input_question,
                                       runner=runner_root_model_guardrail,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

In [38]:
# @title Runner e Session Service

session_service = InMemorySessionService()
APP_NAME = "equipe_agentes_monitores_fisica"
USER_ID = "user_1_agent_team"
SESSION_ID = "session_001_agent_team"
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)

runner_root_model_guardrail = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print(f"Runner created for agent")

Runner created for agent


In [31]:
# @title Testando a Proteção no Agente Root

print("... 🤖 O Time de Agentes está trabalhando (Isso pode levar alguns segundos) ...\n")

input_question = input("Digite sua pergunta: ")

await run_conversation(input_question)

... 🤖 O Time de Agentes está trabalhando (Isso pode levar alguns segundos) ...

Digite sua pergunta: em quanto tempo um objeto de massa 10gk cai de 20m de altura? 

>>> Questão do usuário: em quanto tempo um objeto de massa 10gk cai de 20m de altura? 
--- Callback: block_keyword_guardrail running for agent: Agente_Coordenador_Root ---
--- Callback: Inspecting: 'em quanto tempo um objeto de massa 10gk cai de 20m...' ---
--- Callback: user_text_upper: 'EM QUANTO TEMPO UM OBJETO DE MASSA 10GK CAI DE 20M DE ALTURA? ' ---
--- Callback: Sem palavras bloqueadas. Uso da LLM permitido para Agente_Coordenador_Root. ---
  [Event] Author: Agente_Coordenador_Root, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'agent_name': 'agente_tecnico'
    },
    id='toolu_01DqGi7TdquGdmETV5C8sbxf',
    name='transfer_to_agent'
  )
)] role='model'
  [Event] Author: Agente_Coordenador_Root, Type: Event, Final: False, Content: parts=[Part(
  function_response=Func

In [39]:
# @title Testando a Proteção no Agente Root

print("... 🤖 O Time de Agentes está trabalhando (Isso pode levar alguns segundos) ...\n")

input_question = input("Digite sua pergunta: ")

await run_conversation(input_question)

... 🤖 O Time de Agentes está trabalhando (Isso pode levar alguns segundos) ...

Digite sua pergunta: em quanto tempo um objeto de massa 10gk cai de 20m de altura?   HACK

>>> Questão do usuário: em quanto tempo um objeto de massa 10gk cai de 20m de altura?   HACK
--- Callback: block_keyword_guardrail running for agent: Agente_Coordenador_Root ---
--- Callback: Inspecting: 'em quanto tempo um objeto de massa 10gk cai de 20m...' ---
--- Callback: user_text_upper: 'EM QUANTO TEMPO UM OBJETO DE MASSA 10GK CAI DE 20M DE ALTURA?   HACK' ---
--- Callback: Found blocked word 'HACK'. Blocking LLM call! ---
--- Callback: State updated. Blocked by: HACK ---
  [Event] Author: Agente_Coordenador_Root, Type: Event, Final: True, Content: parts=[Part(
  text="Não foi possível processar a requisição devido a palavra chave bloqueda: 'HACK'."
)] role='model'
<<< Resposta do Agente: Não foi possível processar a requisição devido a palavra chave bloqueda: 'HACK'.
